In [2]:
#import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split

print(tf.__version__)
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()  
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.5, random_state=42) #valid data for hyper params tuning


def return_compiled_model():
    model = tf.keras.models.Sequential()
    #model.add(tf.keras.layers.Flatten())   #Flatten the images! Could be done with numpy reshape
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28))),
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    # model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape= x_train.shape[1:]))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))   #10 because dataset is numbers from 0 - 9
    model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
    return model

model = return_compiled_model()
#model.summary()
# log_dir = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_name = 'main_run_'
log_dir = "logs/fit/" + log_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

with tf.device('/CPU:0'): # it can be with '/CPU:0'
# with tf.device('/GPU:0'): # comment the previous line and uncomment this line to train with a GPU, if available.
    history = model.fit(
        x_train, 
        y_train,
        # batch_size = batch_size,
        epochs=3, 
        # verbose=1,
        shuffle=True,
        # steps_per_epoch = int(normed_train_data.shape[0] / batch_size) ,
        validation_data = (x_valid, y_valid), 
        callbacks=[tensorboard_callback]  
    )


val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy

2.9.0
Epoch 1/3
938/938 [==============================] - 14s 13ms/step - loss: 0.3587 - accuracy: 0.8967 - val_loss: 0.2100 - val_accuracy: 0.9366
Epoch 2/3
938/938 [==============================] - 13s 13ms/step - loss: 0.1494 - accuracy: 0.9542 - val_loss: 0.1473 - val_accuracy: 0.9549
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.1259 - accuracy: 0.9613
0.12588179111480713
0.9613000154495239


In [1]:
#import tensorflow.keras as keras
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time
from sklearn.preprocessing import StandardScaler
import numpy as np # for math and arrays

from DataHandling.DataImporter import DataImporter

all_ds = pd.read_csv('iris_dataset.csv')

all_ds = all_ds.sample(frac=1) # This will randomly shuffle the rows to make sure the data is not sorted

# Split the data into 60% train and 40% test (later will divide the test to test and validate.)
train_dataset, temp_test_dataset =  train_test_split(all_ds, test_size=0.2)
# Split the test_dataset dataframe to 50% test and 50% validation. [this will divide the dataset into 60% train, 20% validate, and 20% test]
test_dataset, valid_dataset =  train_test_split(temp_test_dataset, test_size=0.5)

train_labels1 = train_dataset.pop('target')
test_labels1 = test_dataset.pop('target')
valid_labels1 = valid_dataset.pop('target')

# Encode the labeles
train_labels = pd.get_dummies(train_labels1, prefix='Label')
valid_labels = pd.get_dummies(valid_labels1, prefix='Label')
test_labels = pd.get_dummies(test_labels1, prefix='Label')

# Plot the relationship between each two variables to spot anything incorrect.
#train_stats = train_dataset.describe()
#sns.pairplot(train_stats[train_stats.columns], diag_kind="kde") # or diag_kind='reg'

# Statistics on the train dataset to make sure it is in a good shape. (you may display the same stat for test and validate)
train_stats = train_dataset.describe()
#train_stats.pop("target")
train_stats = train_stats.transpose()

normed_train_data = pd.DataFrame(StandardScaler().fit_transform(train_dataset), columns=train_dataset.columns, index=train_dataset.index)
normed_test_data = pd.DataFrame(StandardScaler().fit_transform(test_dataset), columns=test_dataset.columns, index=test_dataset.index)
normed_valid_data = pd.DataFrame(StandardScaler().fit_transform(valid_dataset), columns=valid_dataset.columns, index=valid_dataset.index)
    

def return_compiled_model(how_many_dense_layers:int, layer_size:int) ->tf.keras.models.Sequential:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten()),
    for i in range(how_many_dense_layers):
        model.add(tf.keras.layers.Dense(layer_size, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))   
    model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
    return model

def create_model_short_name(how_many_dense_layers:int, layer_size:int) ->str:
    return str(how_many_dense_layers)+'layers_'+str(layer_size)+'nodes_'

all_model_specs = {"layers": 0 ,"nodes": 32 }, {"layers": 1 ,"nodes": 32 }, {"layers": 1 ,"nodes": 64 }, {"layers": 1 ,"nodes": 128 }, {"layers": 1 ,"nodes": 256 }, {"layers": 2 ,"nodes": 32 }, {"layers": 2 ,"nodes": 64 }, {"layers": 2 ,"nodes": 128 }, {"layers": 2 ,"nodes": 256 }, {"layers": 3 ,"nodes": 32 }, {"layers": 3 ,"nodes": 64 }, {"layers": 3 ,"nodes": 128 }, {"layers": 3 ,"nodes": 256 }

for model_spec in tqdm(all_model_specs):
    log_name = create_model_short_name(model_spec['layers'],model_spec['nodes'])
    log_dir = "logs_iris/fit/" + log_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model = return_compiled_model(model_spec['layers'],model_spec['nodes'])
    with tf.device('/CPU:0'): # it can be with '/CPU:0'
        history = model.fit(x_train, y_train, epochs=20, shuffle=True, validation_data = (x_valid,y_valid), verbose=0, callbacks=[tensorboard_callback] )

# 1. das nochmal laufen lassen
# 2. das gleiche für iris
# 3. das gleiche für titatnic


2.9.0


  8%|▊         | 1/13 [01:46<21:12, 106.07s/it]